# Test of Autogen Compatibility with Local Inference
Autogen has many tools allowing agent creation

In [30]:
#%pip uninstall autogen
#!py -m pip install -Uqq pyautogen

In [31]:
import autogen

from autogen import AssistantAgent, UserProxyAgent

In [32]:
import os

model_path = "../LLM/TheBloke/Llama-2-7B-Chat-GGUF/llama-2-7b-chat.Q6_K.gguf" # path to your model

if os.path.exists(model_path):
    print(f"{model_path} exists.")
else:
    print(f"{model_path} does not exist.")

../LLM/TheBloke/Llama-2-7B-Chat-GGUF/llama-2-7b-chat.Q6_K.gguf exists.


In [13]:
#Initiate a Llama_cpp server with the model given in model_path defined above
#Server is by default at localhost:8000
#can be modified by using --host 0.0.0.0 and --port to modify ip address and port
!start python -m llama_cpp.server --model {model_path} 

In [43]:
from llama_index.core import PromptTemplate

#Define the prompt_template for Llama2 Chat model
template=("""
<s>[INST] <<SYS>>
{system_prompt}
<</SYS>>

{user_message} [/INST]
""")

qa_template = PromptTemplate(template)

# Example of how to use the qa_template
#prompt = qa_template.format(system_prompt="be nice", user_message="tell me what is Autogen?")

#print(prompt)

In [34]:
config_list = [
    {
        "model":"local-model",
        "api_type": "open_ai",
        "base_url": "http://localhost:8000/v1",
        "api_key": "NULL"
    }
]

llm_config={
#    "request_timeout": 600,
    "timeout":600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0.1,
    "max_tokens":4096,
#    "stream": True,
}

In [35]:
assistant = AssistantAgent(
    name="assistant",
    system_message="""You are a helpful assistant. Provide concise answer.
    When you have answered the question, append TERMINATE to your response and end the chat""",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content","").rstrip().endswith("TERMINATE"),

)

In [39]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    system_message="""Evaluate response and reply TERMINATE if the task has been solved. 
    Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    is_termination_msg=lambda x: x.get("content","").rstrip().endswith("TERMINATE"),
    code_execution_config = {"use_docker": False},
    llm_config = llm_config,

)

In [40]:
assistant.reset()

In [42]:
user_proxy.initiate_chat(
    assistant,
    message=qa_template.format(system_prompt="""Evaluate response and reply TERMINATE if the task has been solved. 
    Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""", 
                               user_message="What are important security aspects when uploading a file in Flask HTML ?"),    
)

user_proxy (to assistant):


<s>[INST] <<SYS>>
Evaluate response and reply TERMINATE if the task has been solved. 
    Otherwise, reply CONTINUE, or the reason why the task is not solved yet.
<</SYS>>

What are important security aspects when uploading a file in Flask HTML ? [/INST]


--------------------------------------------------------------------------------
assistant (to user_proxy):

  CONTINUE:
When uploading a file in Flask HTML, there are several important security aspects to consider:
1. Sanitization: Sanitize user-generated content, including files, to prevent cross-site scripting (XSS) attacks.
2. Validation: Validate file types and sizes to prevent malicious files from being uploaded.
3. Access Control: Implement proper access control mechanisms to restrict access to sensitive files and prevent unauthorized access.
4. Storage: Store files securely, using a secure storage mechanism such as AWS S3 or Google Cloud Storage.
5. Encryption: Encrypt files during storage and tra

ChatResult(chat_history=[{'content': '\n<s>[INST] <<SYS>>\nEvaluate response and reply TERMINATE if the task has been solved. \n    Otherwise, reply CONTINUE, or the reason why the task is not solved yet.\n<</SYS>>\n\nWhat are important security aspects when uploading a file in Flask HTML ? [/INST]\n', 'role': 'assistant'}, {'content': '  CONTINUE:\nWhen uploading a file in Flask HTML, there are several important security aspects to consider:\n1. Sanitization: Sanitize user-generated content, including files, to prevent cross-site scripting (XSS) attacks.\n2. Validation: Validate file types and sizes to prevent malicious files from being uploaded.\n3. Access Control: Implement proper access control mechanisms to restrict access to sensitive files and prevent unauthorized access.\n4. Storage: Store files securely, using a secure storage mechanism such as AWS S3 or Google Cloud Storage.\n5. Encryption: Encrypt files during storage and transmission to protect sensitive data.\n6. Authentic